In [96]:
import numpy as np
import pandas as pd
import numpy as np
import category_encoders as ce

def normalise(a):
    if a<0:
        a=-a
    a=a/np.power(10,np.floor(np.log10(a)))
    return a

    

In [97]:
df = pd.read_csv('train.csv', low_memory=False)
test = pd.read_csv('test.csv', low_memory=False)
test.drop(columns=['Id'],inplace=True)
df = pd.concat([df,test])

df['Location'] = df['Location'].str.replace(",",' ')
# df[['loc1','loc2']] = df.Location.str.split(' ',expand = True).astype(float)
# print(df['loc2'].dtype)
# print(len(df[]))
# for i in range(len(df.index)):
# print(df['loc2'].shape)

df['Crash Date/Time'] = pd.to_datetime(df['Crash Date/Time'])
df['Hour'] = df['Crash Date/Time'].dt.hour
df['day'] = df['Crash Date/Time'].dt.dayofweek
df['Agency Name'] = df['Agency Name'].str.replace("Montgomery County Police",'MONTGOMERY')
df['Driver Substance Abuse'] = df['Driver Substance Abuse'].str.replace("CONTRIBUTED",'PRESENT')
df['Driver Substance Abuse'] = df['Driver Substance Abuse'].str.replace("COMBINED SUBSTANCE",'COMBINATION')
# df['ACRS Report Type'].fillna("UNKNOWN", inplace=True)
df.drop(columns = ['Report Number','Local Case Number','Road Name','Cross-Street Name','Off-Road Description','Municipality','Location','Circumstance','Vehicle Model', 'Vehicle Make','Vehicle ID','Road Name','Person ID','Driverless Vehicle','Crash Date/Time','Vehicle Year'],inplace=True)

# df['Route Type'].fillna("UNKNOWN", inplace=True)
# df['Cross-Street Type'].fillna("UNKNOWN", inplace=True)
# df['Collision Type'].fillna("UNKNOWN", inplace=True)
# df['Parked Vehicle'].fillna("UNKNOWN", inplace=True)
# df['Vehicle First Impact Location'].fillna("UNKNOWN", inplace=True)
# df['Vehicle Second Impact Location'].fillna("UNKNOWN", inplace=True)
# df['Vehicle Damage Extent'].fillna("UNKNOWN", inplace=True)
# df['Injury Severity'].fillna("UNKNOWN", inplace=True)
# df['Vehicle Movement'].fillna("UNKNOWN", inplace=True)
# df['Driver Substance Abuse'].fillna("UNKNOWN", inplace=True)
# df['Non-Motorist Substance Abuse'].fillna("UNKNOWN", inplace=True)
# df['Weather'].fillna("CLEAR", inplace=True)
# df['Surface Condition'].fillna("DRY", inplace=True)
# df['Light'].fillna("DAYLIGHT", inplace=True)
# df['Vehicle Body Type'].fillna("PASSENGER CAR", inplace=True)
# df['Traffic Control'].fillna("UNKNOWN", inplace=True)
# df['Agency Name'].fillna("UNKNOWN", inplace=True)
# df['Vehicle Continuing Dir'].fillna("UNKNOWN", inplace=True)
# df['Vehicle Going Dir'].fillna("UNKNOWN", inplace=True)
# df['Equipment Problems'].fillna("NO MISUSE", inplace=True)
cl=[]
for column in df.columns:
    if df[column].dtype == object:
        cl.append(column)
encoder = ce.polynomial.PolynomialEncoder(verbose=False, cols=cl, drop_invariant=True)
one_hot= encoder.fit_transform(df)

# one_hot = pd.get_dummies(df)
# one_hot.info()
# one_hot.head()
# one_hot['Agency Name_MONTGOMERY'].value_counts(dropna=False)



/home/cmaspi/.local/lib/python3.9/site-packages/category_encoders/polynomial.py:216: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X.drop(col, 1, inplace=True)


In [98]:
label = one_hot['Fault'].to_numpy(dtype=float)
train = one_hot.drop(columns = ['Fault']).to_numpy(dtype=float)
train.shape
# print('hello',df['loc1'][0],df['loc2'][1])#=normalise(df['loc2'][0])
# 

(128725, 316)

In [99]:
print(train.shape)
test = train[51490:]
train = train[:51490]
label = label[:51490]
from sklearn.model_selection import train_test_split

x,xt,y,yt = train_test_split(train, label, test_size=0.2)
test.shape

(128725, 316)


(77235, 316)

In [100]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
base_learners = [
                 ('rf_1', RandomForestClassifier(n_estimators=100, max_features='sqrt')),
                 ('rf_2', LGBMClassifier(boosting_type='dart', drop_rate=0.5, max_drop=50, n_iter=1000)),
                 ('rf_3', XGBClassifier(use_label_encoder=False, learning_rate = 0.14, n_estimators=125, max_depth=8))
                ]
from sklearn.ensemble import StackingClassifier
# clf = LGBMClassifier(boosting_type='dart', drop_rate=0.5, max_drop=50, n_iter=1000)
# clf = StackingClassifier(estimators=base_learners, n_jobs=12, final_estimator=LGBMClassifier(boosting_type='goss'))
# clf.fit(train , label)

# from sklearn import model_selection
# from sklearn.ensemble import BaggingClassifier
# seed = 8
base_cls = StackingClassifier(estimators=base_learners, n_jobs=12, final_estimator=LGBMClassifier(boosting_type='goss'))
# # bagging classifier
# clf = BaggingClassifier(base_estimator = base_cls,
#                           random_state = seed, max_samples=0.1,n_estimators=1)
  
# clf.fit(train, label)
n_iter = 50
clf=[]
for _ in range(n_iter):
    c = StackingClassifier(estimators=base_learners, n_jobs=12)
    _,train1,_,label1 = train_test_split(train, label, test_size=0.05)
    c.fit(train1,label1)
    clf.append(c)



# pred = clf.predict(xt)
# print(yt.shape, pred.shape)
# print(accuracy(yt, pred))
# print(yt, pred)

KeyboardInterrupt: 

In [ ]:
# clf.score(xt, yt)


In [ ]:
pred = []
p=[]
for j in range(n_iter):
    classifier = clf[j]
    pred.append(classifier.predict(test))
pred = np.array(pred)
print(len(pred[0]))
print(pred[:5])

# n = len(pred)
# pred =np.c_[np.linspace(0,n-1,n),pred]
# pred=np.array(pred, dtype=int)
# output = pd.DataFrame(pred)
# output.to_csv('output.csv', header= ['Id','Fault'],index=False)
# output.info()

77235
[[1. 0. 0. ... 0. 1. 1.]
 [1. 0. 1. ... 0. 1. 1.]
 [1. 0. 1. ... 0. 1. 1.]
 [1. 0. 0. ... 0. 1. 1.]
 [1. 0. 0. ... 0. 1. 1.]]


In [ ]:
print(pred[:,1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [ ]:
from scipy.stats import mode
predicted = []
for i in range(len(pred[0])):
    predicted.append(mode( pred[:,i] ))
predicted = np.array(predicted)
pred = predicted[:,0]

In [ ]:
n = len(pred)
print(n)
pred =np.c_[np.linspace(0,n-1,n),pred]
pred=np.array(pred, dtype=int)
output = pd.DataFrame(pred)
output.to_csv('output.csv', header= ['Id','Fault'],index=False)
output.info()

77235
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77235 entries, 0 to 77234
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       77235 non-null  int64
 1   1       77235 non-null  int64
dtypes: int64(2)
memory usage: 1.2 MB
